# api 크롤링

기존의 크롤링 방식은

1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나 (selenium)

혹은

2. 파이썬 자체적으로 호출을 요청해서 (requests)

데이터를 가지고 오는 방식이었습니다

이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다

따라서 서비스 제공자측에서는 서버 부하를 줄이고, 사용자에게 맞춤형 데이터를 제공하기 위해 api서버를 운영합니다

api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 있습니다)
(라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다)
필요없는 동영상자료나 그림자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다


api 서버 접근시 보통 urllib.request를 이용하게 됩니다

In [3]:
# 사이트에 자료 요청
import urllib.request
# json 데이터 핸들링
import json
# DataFrame 자료형 활용
import pandas as pd
# json 데이터를 pandasDataFrame 변환
from pandas.io.json import json_normalize

# 영진위 api신청

http://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

1. 접속 후 가입 및 로그인

2. 키발급 텝에서 키 발급하기

3. 요청 api종류 보고 요청양식 및 데이터 확인하기

In [4]:
api_key = "cb38ebad17f6f72f4ea682c05a8bc8df"
request_data = "20220622"
url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_data)
print(url)

https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220622


# json 데이터를 팬더스 데이터프레임으로 변환
- api데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다
- 쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데
- 파이썬의 딕셔너리와 거의 같다고 보시면 됩니다
- 따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상
- 딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [5]:
# 위의 url 변수를 이용해 데이터를 요청합니다
r = urllib.request.urlopen(url)

# 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다
json_raw = r.read()

In [6]:
# b'~~~~~~~~~~'로 시작하는 이 데이터는 파이썬으로 byte자료형으로 처리되고
# 데이터프레임으로는 바로 변환이 불가능합니다
json_utf8 = json_raw.decode("utf-8")

In [7]:
json_utf8

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20220622~20220622","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"NEW","movieCd":"20194376","movieNm":"탑건: 매버릭","openDt":"2022-06-22","salesAmt":"1976049530","salesShare":"54.3","salesInten":"1976049530","salesChange":"100","salesAcc":"2716938360","audiCnt":"188312","audiInten":"188312","audiChange":"100","audiAcc":"253016","scrnCnt":"1975","showCnt":"9158"},{"rnum":"2","rank":"2","rankInten":"-1","rankOldAndNew":"OLD","movieCd":"20224882","movieNm":"마녀(魔女) Part2. The Other One","openDt":"2022-06-15","salesAmt":"836476290","salesShare":"23.0","salesInten":"-253649090","salesChange":"-23.3","salesAcc":"18226409650","audiCnt":"82691","audiInten":"-25491","audiChange":"-23.6","audiAcc":"1769461","scrnCnt":"976","showCnt":"3502"},{"rnum":"3","rank":"3","rankInten":"-1","rankOldAndNew":"OLD","movieCd":"20204548","movieNm":"범죄도시 2","openDt":"2022-05-18","salesAmt":"577713370","salesShare":"15.9","sale

In [8]:
# json.loads(딕셔너리 형태의 문자열 자료)
# 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환
json_complete = json.loads(json_utf8)

In [9]:
# 딕셔너리(json)변환 완료 확인
type(json_complete)

dict

In [10]:
# json_normalize(딕셔너리 자료)
# 딕셔너리 자료를 데이터프레임 형태로 변환해줌
pd.json_normalize(json_complete)

,boxOfficeResult.boxofficeType,boxOfficeResult.showRange,boxOfficeResult.dailyBoxOfficeList
0,일별 박스오피스,20220622~20220622,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."


In [11]:
# boxOfficeResult.dailyBoxOfficeList 컬럼 내부에 딕셔너리가 여럿 묶인 리스트가 보이므로
# boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다
box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

In [12]:
# json_normalize()는 pd.DataFrame()으로 대체 가능합니다
pd.DataFrame(json_complete)

,boxOfficeResult
boxofficeType,일별 박스오피스
dailyBoxOfficeList,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."
showRange,20220622~20220622


In [13]:
url

'https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220622'

api 크롤링으로 서비스 제공시

사용자가 조회할때마다 매번 API서버에 요청을 넣어서 데이터를 가져오면

호출횟수를 금방 소모하게 됩니다

따라서 , 보통 API를 활용한 서비스는 미리 데이터를 DB에 적재해놓고

사용자가 요청시 내 DB에 있는 데이터를 우선적으로 보여주고, 없으면 외부서버에 

요청해서  내 DB에 INSERT한 다음 다시 내 DB데이터를 사용자에게 보여줍니다

날짜 데이터는 28일까지인 달 (2월), 30일까지인 달, 31일까지인 달 들이 있어서

단순히 문자열에 1씩 더하거나 날짜처리를 하기도 어렵고

위의 처리를 다 했다고 해도 4년에 1번 윤달이 있습니다(2월 29일)

그래서, 날짜는 전적으로 날짜데이터를 이용해 처리해야만 합니다

파이썬에서는 datetime을 이용해 처리합니다.(자바는 java.util.Date를 써서 처리)

In [14]:
import time
import datetime # 파이썬 날짜 자료형

In [15]:
start_datetime = datetime.datetime(2021, 8, 9) # 년 월 일 순으로 넣어줍니다
# 형식 바꾸기
target_date = start_datetime.strftime('%Y%m%d') # 영진위 형식으로  변경
print(target_date)
next_datetime = start_datetime + datetime.timedelta(days=1)
next_datetime

20210809


datetime.datetime(2021, 8, 10, 0, 0)

In [16]:
target_dates = ['20220805','20220806','20220807']
# 날짜는 시작날짜와 + 며칠을 처리할지 결정할 수 있습니다
# 3일치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요

# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요
# target_date 컬럼에 조회 날짜를 추가해주세요
# 빈 데이터프레임을 하나 생성해놓고(pd.DataFrame())
merged_data = pd.DataFrame()
for date in target_dates:
    api_key = "cb38ebad17f6f72f4ea682c05a8bc8df"
    request_data = date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_data)
    print(url)
    rr = urllib.request.urlopen(url)
    json_raws = rr.read();
    json_utf8 = json_raws.decode("utf-8")
    json_complete = json.loads(json_utf8)
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['날짜'] = date
    merged_data = pd.concat([merged_data, box_result])
merged_data

https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220805
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220806
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220807


,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,1,OLD,20209343,한산: 용의 출현,2022-07-27,2565562184,41.6,530997656,26.1,37567526644,250628,41281,19.7,3692259,1530,6343,20220805
1,2,2,-1,OLD,20196410,비상선언,2022-08-03,2094794151,34.0,71338791,3.5,7693545482,213574,-8542,-3.8,794759,1718,7281,20220805
2,3,3,1,OLD,20194376,탑건: 매버릭,2022-06-22,504483247,8.2,148299938,41.6,77905793874,46942,12422,36,7305829,605,1238,20220805
3,4,4,-1,OLD,20205362,미니언즈2,2022-07-20,413915957,6.7,948579,0.2,17793954400,43092,-1769,-3.9,1837970,783,1595,20220805
4,5,5,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,167553618,2.7,-5960802,-3.4,2909201168,18088,-1181,-6.1,312147,537,779,20220805
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,142017614,2.3,34703455,32.3,18014179742,13226,3017,29.6,1741846,265,432,20220805
6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,113278760,1.8,22370761,24.6,15283607796,11557,2267,24.4,1466378,407,576,20220805
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,50015320,0.8,-4063959,-7.5,192992299,5268,-678,-11.4,20861,362,513,20220805
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,27421500,0.4,3966180,16.9,4509968774,2517,232,10.2,434341,75,94,20220805
9,10,10,26,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,12398000,0.2,11939500,2604,28919650670,2433,2378,4323.6,2806159,20,20,20220805


In [17]:
start_datetime = datetime.datetime(2021, 8, 9)

# 날짜는 시작날짜와 + 며칠을 처리할지 결정할 수 있습니다
# 1년치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요

# 반복문과 append를 이용해서 1년치 데이터를 요청해서 합쳐보세요
# target_date 컬럼에 조회 날짜를 추가해주세요
# 빈 데이터프레임을 하나 생성해놓고(pd.DataFrame())
merged_data = pd.DataFrame()
for idx in range(1, 366):
 # 년 월 일 순으로 넣어줍니다
    # 형식 바꾸기
    target_date = start_datetime.strftime('%Y%m%d') # 영진위 형식으로  변경
    print(target_date)

    api_key = "cb38ebad17f6f72f4ea682c05a8bc8df"

    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, target_date)
    print(url)
    rr = urllib.request.urlopen(url)
    json_raws = rr.read();
    json_utf8 = json_raws.decode("utf-8")
    json_complete = json.loads(json_utf8)
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['날짜'] = date
    merged_data = pd.concat([merged_data, box_result])
    start_datetime = start_datetime + datetime.timedelta(days=1)
    time.sleep(1)
merged_data

20210809
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20210809
20210810
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20210810
20210811
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20210811
20210812
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20210812
20210813
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20210813
20210814
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20210814
20210815
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

20211002
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211002
20211003
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211003
20211004
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211004
20211005
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211005
20211006
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211006
20211007
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211007
20211008
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

20211125
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211125
20211126
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211126
20211127
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211127
20211128
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211128
20211129
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211129
20211130
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20211130
20211201
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

20220118
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220118
20220119
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220119
20220120
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220120
20220121
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220121
20220122
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220122
20220123
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220123
20220124
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

20220313
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220313
20220314
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220314
20220315
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220315
20220316
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220316
20220317
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220317
20220318
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220318
20220319
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

20220506
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220506
20220507
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220507
20220508
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220508
20220509
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220509
20220510
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220510
20220511
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220511
20220512
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

20220629
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220629
20220630
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220630
20220701
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220701
20220702
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220702
20220703
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220703
20220704
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=cb38ebad17f6f72f4ea682c05a8bc8df&targetDt=20220704
20220705
https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBox

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,709693260,55.3,-1291136120,-64.5,17288767790,75348,-121324,-61.7,1787723,1559,6247,20220807
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,248909890,19.4,-427887350,-63.2,3191163840,24477,-38618,-61.2,305854,962,3007,20220807
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,84519500,6.6,-199686520,-70.3,7595918080,9547,-20499,-68.2,846352,655,1315,20220807
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,74063300,5.8,-120426480,-61.9,29531394670,7568,-11172,-59.6,2916247,494,873,20220807
4,5,5,1,OLD,20218349,정글 크루즈,2021-07-28,28406200,2.2,-68337650,-70.6,2228562490,3150,-6675,-67.9,241040,426,569,20220807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,6,-1,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,63100299,1.9,-201077501,-76.1,3498746166,6967,-20624,-74.7,374124,456,679,20220807
6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,49356700,1.5,-74608418,-60.2,15679219151,4842,-6848,-58.6,1501777,351,485,20220807
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,22953320,0.7,-54773879,-70.5,369229618,2534,-5369,-67.9,38953,315,373,20220807
8,9,9,0,NEW,20224965,DC 리그 오브 슈퍼-펫,2022-08-10,20148000,0.6,20148000,100,24405000,2117,2117,100,2590,9,11,20220807


In [18]:
# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요
#관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다
# 컬럼 타입이 object 이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다
# 현재 수치데이터도 전부 문자로 간주되고 있기 때문에 int로 자료형을 바꿔서 계산해야함
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int)

In [19]:
result = merged_data.pivot_table(index=['movieNm'], aggfunc='sum', values=['scrnCnt','audiCnt'])

In [20]:
result['scrnMean'] = result['audiCnt'] / result['scrnCnt']

In [21]:
result[result['scrnCnt'] > 100].sort_values('scrnMean', ascending=False).head(30)

,audiCnt,scrnCnt,scrnMean
movieNm,,,
한산: 용의 출현,4759478,24231,196.421031
범죄도시 2,12686014,81645,155.380170
비상선언,1495293,9971,149.964196
탑건: 매버릭,7477418,58469,127.886880
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
쥬라기 월드: 도미니언,2832531,24893,113.788254
마녀(魔女) Part2. The Other One,2795796,27322,102.327648
토르: 러브 앤 썬더,2711863,26516,102.272703
미니언즈2,1994072,20169,98.868164


In [22]:
# 스크린당 관객수가 평균이상이면서 스크린 개수가 50개 이상인 영화만 나열해주세요
final_data = result[(result['scrnCnt'] >= 50) & (result['scrnMean'] >= 26.5)].head(50)

In [23]:
result[result['scrnCnt'] >= 50]['scrnMean'].mean()

26.589802739224456

In [24]:
# csv로 export하기
final_data.to_csv('평균이상영화.csv', encoding="utf-8")

In [25]:
# excel로 export하기
final_data.to_excel('평균이상영화.xlsx', encoding="utf-8")

PermissionError: [Errno 13] Permission denied: '평균이상영화.xlsx'

# 가져온 데이터 mysql에 연동하기

- import pymysql
- 을 활용해서 연동할 수 잇습니다

- 임포트가 되지 않는 다면 다시 설치를 해야합니다
- 1. Anaconda Navigator에서 좌측 탭의 enviroments를 누릅니다
- 2. Open terminal을 이용해 cmd창 같은 창을 켭니다(터미널창이라고 부릅니다
- 3. pip install pymysql 을 입력하고 실행합니다

### 오라클 연동

위와 같은데 3번에서
- pip install cx.Oracle
- 을 대신 입력해서 설치하면 됩니다.

In [26]:
import pymysql

In [27]:
# MySql 연결에 필요한 정보를 입력합니다
con = pymysql.connect(host='localhost', user='root', password='mysql', db='pyprac2', charset='utf8',
                     cursorclass=pymysql.cursors.DictCursor) # 컬럼명까지 받아올지 여부

In [28]:
# SQL구문을 받아주는 cursor 객체를 사용합니다
cur = con.cursor()

In [29]:
# SQL구문은 문자로 cur.execute()를 실행해서 수행시킵니다
cur.execute("SELECT * FROM test_tbl2")

7

In [30]:
# execute가 실행되면, cur가 데이터를 저장하고 있는데이걸 파이썬 형식으로 빼야합니다
mysql_data = cur.fetchall()
mysql_data

[{'tno': 1, 'tname': 'a'},
 {'tno': 2, 'tname': 'b'},
 {'tno': 3, 'tname': 'c'},
 {'tno': 4, 'tname': 'd'},
 {'tno': 5, 'tname': 'e'},
 {'tno': 6, 'tname': 'f'},
 {'tno': 7, 'tname': 'g'}]

In [31]:
# 데이터프레임 형태로 변환
pd.DataFrame(mysql_data)

,tno,tname
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,f
6,7,g


In [32]:
cur.execute("INSERT INTO test_tbl2(tname) VALUES ('g')")

1

In [33]:
# 커서가 아닌 커넥션에 commit을 주는것에 주의해주세요
con.commit()

In [34]:
cur.execute("SELECT * FROM test_tbl2")

8

In [35]:
mysql_data = cur.fetchall()

In [36]:
mysql_data

[{'tno': 1, 'tname': 'a'},
 {'tno': 2, 'tname': 'b'},
 {'tno': 3, 'tname': 'c'},
 {'tno': 4, 'tname': 'd'},
 {'tno': 5, 'tname': 'e'},
 {'tno': 6, 'tname': 'f'},
 {'tno': 7, 'tname': 'g'},
 {'tno': 8, 'tname': 'g'}]

In [37]:
pd.DataFrame(mysql_data)

,tno,tname
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,f
6,7,g
7,8,g


In [38]:
merged_data.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,709693260,55.3,-1291136120,-64.5,17288767790,75348,-121324,-61.7,1787723,1559,6247,20220807
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,248909890,19.4,-427887350,-63.2,3191163840,24477,-38618,-61.2,305854,962,3007,20220807
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,84519500,6.6,-199686520,-70.3,7595918080,9547,-20499,-68.2,846352,655,1315,20220807
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,74063300,5.8,-120426480,-61.9,29531394670,7568,-11172,-59.6,2916247,494,873,20220807
4,5,5,1,OLD,20218349,정글 크루즈,2021-07-28,28406200,2.2,-68337650,-70.6,2228562490,3150,-6675,-67.9,241040,426,569,20220807


In [39]:
# 로우개수
len(merged_data) # 3650번 반복해야 DB에 insert가 된다는걸 알 수 있음
# iloc[], .loc[] 를 이용하면 특정 로우자료를 조회할 수 있고, 거기서 ['컬럼명']을 집어넣으면
# 특정 로우의 특정 컬럼값을 조회할 수 있습니다
merged_data.iloc[4]['movieNm']

'정글 크루즈'

In [40]:
# 과제, MySQL이나 Oracle SQL에 현재 보고있는 db_insert 데이터를
# 테이블 생성 및 반복문을 이용해서 다 DB에 넣어주시면 됩니다
# 반복문을 이용해서 merged_data의 모든 데이터를 MySQL에 집어넣어주세요
# 집어넣을 컬럼은 rnum, movieNm, openDt, audiCnt, showCnt, date

In [75]:
merged_data['openDt']

0    2021-07-28
1    2021-08-04
2    2021-07-21
3    2021-07-07
4    2021-07-28
        ...    
5    2022-07-28
6    2022-07-20
7    2022-08-03
8    2022-08-10
9    2022-07-13
Name: openDt, Length: 3650, dtype: object

In [77]:
for i in range(len(merged_data)):
    selected_data = merged_data.iloc[i]
    cur.execute("INSERT INTO moviechart(movieNm, openDt, audiCnt, showCnt, dataDate) VALUES ('%s',(STR_TO_DATE('%s','%%d-%%m-%%y')), %s, %s, '%s')" %  (selected_data['movieNm'], selected_data['openDt'], selected_data['audiCnt'], selected_data['showCnt'], selected_data['날짜']))

OperationalError: (1411, "Incorrect datetime value: '2021-07-28' for function str_to_date")

In [ ]:
con.close()

In [61]:
for i in range(len(merged_data)):
    print(merged_data.iloc[i]['openDt'])


2021-07-28
2021-08-04
2021-07-21
2021-07-07
2021-07-28
2021-08-11
2021-08-05
2021-07-28
2021-08-05
2021-07-14
2021-07-28
2021-08-04
2021-07-21
2021-07-07
2021-08-05
2021-07-28
2021-08-11
2021-07-28
2021-08-05
2021-07-14
2021-08-11
2021-07-28
2021-08-11
2021-08-04
2021-07-21
2021-07-07
2021-08-05
2021-08-05
2021-08-04
2021-08-19
2021-08-11
2021-07-28
2021-08-11
2021-08-04
2020-01-23
2021-07-21
2021-07-07
2021-08-05
2021-06-30
2021-08-05
2021-08-11
2021-07-28
2021-08-11
2021-08-04
2021-07-21
2021-07-07
2021-08-18
2020-01-23
2021-08-05
2021-06-09
2021-08-11
2021-07-28
2021-08-11
2021-08-04
2021-07-21
2021-07-07
2021-08-05
2020-01-23
2021-06-23
2021-08-18
2021-08-11
2021-07-28
2021-08-11
2021-08-04
2021-07-21
2021-07-07
2021-08-05
2021-08-18
2021-08-05
2020-01-23
2021-08-11
2021-07-28
2021-08-11
2021-07-21
2021-08-04
2021-07-07
2021-08-05
2021-08-05
2020-01-23
2021-07-28
2021-08-11
2021-07-28
2021-08-11
2021-08-04
2021-07-21
2021-07-07
2021-08-05
2021-08-05
2021-08-19
2021-08-31
2021-08-18

2021-11-17
2021-11-24
2021-11-24
2021-11-24
2021-12-01
2021-10-20
2021-11-03
2021-12-01
2021-12-01
2021-11-17
2021-12-01
2021-11-24
2021-11-24
2021-11-24
2021-10-20
2021-12-01
2021-12-01
2021-11-03
2021-12-01
2021-11-17
2021-12-01
2021-11-24
2021-11-24
2021-11-24
2021-12-01
2021-10-20
2021-12-01
2021-12-01
2021-11-03
2021-11-17
2021-12-01
2021-11-24
2021-11-24
2021-12-08
2021-11-24
2021-12-08
2021-10-20
2021-12-01
2021-12-01
2021-11-03
2021-11-17
2021-11-24
2021-11-24
2021-11-24
2021-12-08
2021-12-09
2021-12-08
2021-10-20
2021-12-09
2021-12-01
2021-12-01
2021-11-24
2021-11-24
2021-11-24
2021-12-08
2021-10-20
2021-12-01
2021-12-08
2021-12-09
2021-11-03
2021-12-09
2021-11-24
2021-11-24
2021-11-24
2021-12-08
2021-10-20
2021-12-01
2021-12-08
2021-12-09
2021-11-03
2021-12-01
2021-11-24
2021-11-24
2021-11-24
2021-12-08
2021-10-20
2021-12-01
2021-12-08
2021-11-03
2021-12-09
2021-12-09
2021-11-24
2021-11-24
2021-11-24
2021-12-08
2022-01-26
2021-10-20
2021-12-08
2021-11-03
2021-12-01
2021-12-01

2022-02-17
2022-03-23
2022-04-13
2022-04-06
2022-04-14
2022-04-13
2022-04-13
2022-03-30
2022-04-06
2022-04-06
2022-04-22
2022-04-06
2022-04-27
2022-02-17
2022-03-23
2022-04-13
2022-04-06
2022-03-30
2022-04-06
2022-04-06
2022-04-20
2022-04-13
2022-03-23
2022-02-17
2022-04-06
2022-04-13
2022-04-20
2022-04-20
1995-09-02
2022-04-20
2022-04-06
2022-04-27
2022-03-30
2022-01-26
2022-02-23
2022-04-13
2022-04-20
2022-04-20
2021-01-27
2022-04-20
1995-09-02
2022-04-06
2022-04-21
2022-04-21
2022-04-21
2022-04-13
2022-04-22
2022-04-20
2022-04-20
2021-01-27
2022-04-20
1995-09-02
2022-04-06
2022-04-21
2022-04-21
2022-04-13
2022-04-22
2022-04-20
2022-04-06
2022-04-20
2022-04-20
2021-01-27
2022-05-04
2022-02-17
1995-09-02
2022-04-13
2022-04-22
2022-04-20
2022-04-06
2022-04-20
2022-04-20
2021-01-27
1995-09-02
2022-02-17
2022-03-30
2022-04-13
2022-04-22
2022-04-20
2022-04-20
2022-04-06
2022-04-20
2021-01-27
2022-04-27
1995-09-02
2022-01-26
2022-04-13
2022-04-22
2022-04-20
2022-04-20
2022-04-06
2022-04-27

In [47]:
merged_data[0]['movieNm']

KeyError: 0

3650